In [ ]:
%load_ext autoreload
%autoreload 2
%load_ext dotenv

# Get the output from the cat command and process it
!cp .env .env.tmp
!op inject --force --in-file .env.tmp --out-file .env.tmp
%dotenv -o .env.tmp
!rm .env.tmp

In [ ]:
# import rich
# import os
# rich.print(os.environ)

In [ ]:
import textwrap
from datetime import date, datetime, time, timedelta
from io import StringIO

from loguru import logger
from rich.console import Console

from lmnop_wakeup.common import get_hass_api_key
from lmnop_wakeup.tools.calendars import get_filtered_calendars_with_notes
from lmnop_wakeup.utils.date import format_time_info

start_ts = (
  datetime.combine(date.today() + timedelta(days=1), time.min)
  .astimezone()
  .replace(hour=0, minute=0, second=0)
)
end_ts = (
  datetime.combine(start_ts.date() + timedelta(days=13), time.max)
  .astimezone()
  .replace(hour=23, minute=59, second=59)
)


async def get_calendars():
  try:
    return await get_filtered_calendars_with_notes(
      start_ts=start_ts,
      end_ts=end_ts,
      hass_api_token=get_hass_api_key(),
    )
  except Exception:
    logger.exception("Error while getting calendars")
    raise


calendars = await get_calendars()

In [ ]:
sb = StringIO()
for cal in calendars:
  if not cal.events:
    continue

  sb.write(
    textwrap.dedent(f"""
    --------------------------------

    ## {cal.name}

    > {cal.notes_for_processing}


    """).lstrip()
  )

  for event in cal.events:
    event_time = format_time_info(event.start_ts, "%Y-%m-%d", "%H:%M:%S")
    if event.end_ts:
      event_time += " to "
      event_time += textwrap.dedent(format_time_info(event.end_ts, "%Y-%m-%d", "%H:%M:%S"))
    event_time += ""

    sb.write(
      textwrap.dedent(f"""
      * ### {event.summary}

        {event_time}

      """).lstrip()
    )

    if event.description:
      sb.write("  **Description:**\n")
      sb.write(f"  {event.description}\n\n")

    if event.creator:
      sb.write(f"  **Creator:** {event.creator.email}\n\n")

    if event.attendees:
      sb.write("  **Attendees:**\n")
      for attendee in event.attendees:
        sb.write(f"  - {attendee.email}\n")
      sb.write("\n")

    sb.write("\n\n")

console = Console(width=80)
console.print(sb.getvalue())